In [20]:
using NBInclude
using GLPK
using JuMP
using LinearAlgebra
@nbinclude("parser.ipynb")

# Rota mais valiosa
Instância Um grafo completo G = (V, A) com distâncias da, a ∈ A, entre os vértices, prêmios pv,
v ∈ V , nos vértices e um vértice de origem vo ∈ V .
Solução Uma rota R = (vo, v1, v2, . . . , vk, vo) inciando e terminando em vo com vértices intermediários vi ∈ V . Cada vértice em V \ {vo} pode ser contido no máximo um vez em R.
Objetivo Minimizar o valor total v(R) = dvo,v1 +
∑
i∈[k−1] dvi,vi+1 + dvk,vo −
∑
i∈[k]
pvi da rota.

## Variáveis
- D -> Matriz de adjacência com o peso de cada aresta 
- A -> Matriz de adjacência com a booleano informando se a aresta pertence ou não a solução
- P -> Lista com o peso (prêmio) de cada vértice
- V -> Lista com a booleano informando se o vértice foi visitado ou não

## Modelagem
Minimizar 
    $\sum_{i=0}^{V-1}\sum_{j=0}^{V-1} D_{(i,j)} * A_{(i,j)} - \sum_{i=0}^{V-1} P_{(i)} * V_{(i)}$


Sujeito:
    
 - A aresta i,j pode (1) ou não (0) estar contido na solução: 
     
     $A_{(i,j)} \in \{0,1\}$ 
 
 
 - O vértica i pode (1) ou não (0) estar contido em algumas das arestas da solução:
     
     $V_{(i)} \in \{0,1\}$ 
 
 
 - A aresta que sai do vértice 0, que é vértice de partida, deve obrigatoriamente estar contido na solução
     
     $\sum_{j=1}^{V} A_{(0,j)} = 1$
 
 
 - A aresta que chega no vértice 0, que é vértice de destino, deve obrigatoriamente estar contido na solução
     
     $\sum_{i=1}^{V} A_{(i,1)} = 1$
 
 
 - Vinculação dos vértice visitados com os prêmios destes.
     
     $\sum_{j=1}^{V} A_{(i,j)} = V_{i}, \forall_{i} \in \{1,..,V\}$
 
 
 - Um vértice não pode estar contido como destino em mais de 1 aresta da solução
     
     $\sum_{i=1}^{V-1} A_{(i,j)} <= 1, \forall_{j} \in \{1,..,V\}$
 
 
 - Um vértice não pode estar contido como partida em mais de 1 aresta da solução
     
     $\sum_{j=1}^{V} A_{(i,j)} <= 1, \forall_{i} \in \{1,..,V\}$
 
 
 - Se um vértice está contido como partida em alguma aresta da solução, ele obrigatoriamente deve estar contido como destino em alguma aresta da solução 
     
     $\sum_{i=1}^{V}\sum_{j=1}^{V} A_{(i,j)} - A_{(j, i)} = 0  $

## Código

In [2]:
file = "instancias/A-n32-k5.vrp"
v, P = parse_file(file)
D = get_distancias(v)
cardinalidade = length(P);

In [29]:
m = Model(GLPK.Optimizer)

@variable(m, A[1:cardinalidade, 1:cardinalidade], Bin)
@variable(m, V[1:cardinalidade], Bin);

@objective(m, Min, sum(D .* A) - sum(P .* V));

In [31]:
#Diagonal(A) .== zeros()

In [32]:
@constraints(m, begin
    sum(A[1, :]) == Int8(1)
    sum(A[:, 1]) == Int8(1)
    sum(A, dims=2) .== V
    sum(A, dims=2) .<= ones(Int64, cardinalidade)
    sum(A, dims=1) .<= ones(Int64, (1, cardinalidade))
    sum(A - transpose(A)) == 0
    end
)

In [33]:
optimize!(m)


In [34]:
@show objective_value(m)


objective_value(m) = -410.0


-410.0

In [36]:
JuMP.value.(A)

32×32 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 